In [1]:
import tensorflow as tf
from tensorflow import keras

In [22]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self,filters,strides=1,activation='relu',**kwargs):
        super().__init__(**kwargs) # 그외 매개변수를 사용할 수 있도록 함(필수)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters,3,strides=strides,padding='same',use_bias=False), # 여기서 스트라이드는 필터 갯수의 변동에 따라 유동적임
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters,3,strides=1,padding='same',use_bias=False), # 여기서 스트라이드는 고정시켜줘야 함
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = [] # stides가 1일 때는 그대로 인풋이 더해지도록 하기 위함
        if strides > 1 : # 필터의 갯수가 바뀔 때 스트라이드가 2가 됨. 그리고 스킵연결은 다음과 같이 변함
            self.skip_layers = [
                keras.layers.Conv2D(filters,1,strides=strides,padding='same',use_bias=False),
                keras.layers.BatchNormalization()
            ]
    def call(self, inputs):
        Z = inputs
        for layers in self.main_layers:
            Z = layers(Z)
        skip_Z  = inputs
        for layers in self.skip_layers:
            skip_Z = layers(skip_Z)
        return self.activation(Z + skip_Z)

In [23]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64,7,strides=2,padding='same',input_shape=[32,32,3],use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=3,strides=2,padding='same'))

prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    # 필터의 크기가 변할 때 잔차유닛 첫번째 레이어의 스트라이드가 2가 됨. 동시에 스킵연결 시에 스킵연결을 거치게 됨
    strides= 1 if prev_filters == filters else 2 
    model.add(ResidualUnit(filters, strides = strides)) 
    prev_filters = filters

model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1000))
model.add(keras.layers.Dense(10, activation='softmax'))

In [24]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_108 (Conv2D)         (None, 16, 16, 64)        9408      
                                                                 
 batch_normalization_107 (Ba  (None, 16, 16, 64)       256       
 tchNormalization)                                               
                                                                 
 activation_9 (Activation)   (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 residual_unit_45 (ResidualU  (None, 8, 8, 64)         74240     
 nit)                                                            
                                                     